The present program reads the data; calculates many network parameter; tries to generate the equivalent E-R random network using the average degree, sex of the nodes, and number of nodes; and generates the synthetics statistics for the random network for comparison with the real networks.

In [170]:
### Imports ###
%matplotlib inline
%config IPCompleter.greedy=True
import matplotlib.pyplot as plt
import igraph
import numpy as np
from IPython.display import *
import pandas as pd
from igraph import ADJ_DIRECTED
import os
from igraph import *

In [171]:
### Initialization ###
def displayNetwork(g, visual_style={}):
    """
    IN: Network object
    OUT: Plots the network
    """
    igraph.plot(g, "temp.png", **visual_style)
    display(Image(filename='temp.png'))

### Loading the data to network object###
def LoadData2Network(file_id,it="w1"):
    """
    IN: File_id, Iteration
    OUT: Friendship_Network, Trust_Network
    """
    sex        =pd.read_csv(DATA_PATH+"{}_sex.csv".format(file_id),index_col=0)
    friendship =pd.read_csv(DATA_PATH+"{}_affective_{}.csv".format(file_id,it),index_col=0)
    trust      =pd.read_csv(DATA_PATH+"{}_trust_{}.csv".format(file_id,it),index_col=0)
    A_fr = friendship.values
    A_tr = trust.values
    friend_graph=igraph.Graph.Weighted_Adjacency((A_fr).tolist(),mode=ADJ_DIRECTED, attr="weight", loops=False)
    trust_graph=igraph.Graph.Weighted_Adjacency((A_tr).tolist(),mode=ADJ_DIRECTED, attr="weight", loops=False)
    friend_graph.vs['sex'] = sex['sex'].values
    trust_graph.vs['sex'] = sex['sex'].values
    return friend_graph, trust_graph

In [172]:
### Paths ###
DATA_PATH="Data/"

In [173]:
### Visualisation ###
visual_style = {}
visual_style["layout"] = friend_graph.layout_fruchterman_reingold()
visual_style["margin"] = 60
visual_style["edge_curved"] = 0.2
visual_style["edge_width"] = 0.2
visual_style["edge_arrow_size"] = 0.2
visual_style["edge_arrow_width"] = 1
visual_style["color"]=friend_graph.vs["sex"]

friend_graph.vs["sex"] = friend_graph.vs["sex"]
color_dict = {1: "blue", 2: "red"}
friend_graph.vs["color"] = [color_dict[gender] for gender in friend_graph.vs["sex"]]

displayNetwork(friend_graph,visual_style)

TypeError: plotting not available

In [156]:
### Network parameters ###

def get_network_param(network):
    """
    IN: Network object
    OUT: Bunch of network parameters
    
    """
    Indegree=getattr(network,"")
    Degree =getattr(network,"degree")()
    Betweenness =getattr(network,"betweenness")(vertices=None, directed=True, cutoff=None, weights=None, nobigint=True)
    Transitivity_avglocal_undirected =getattr(network,"transitivity_avglocal_undirected")(mode="nan")
    Transitivity_undirected =getattr(network,"transitivity_undirected")(mode="nan")
    Transitivity_local_undirected =getattr(network,"transitivity_local_undirected")(vertices=None, mode="nan", weights=None)
    Diameter =getattr(network,"diameter")(directed=True, unconn=True, weights=None)
    Density =getattr(network,"density")(loops=False)
    Diversity =getattr(network,"diversity")(vertices=None, weights=None)
    Triad_census =getattr(network,"triad_census")()
    Vertex_connectivity =getattr(network,"vertex_connectivity")(source=-1, target=-1, checks=True, neighbors="error")
    Similarity_jaccard =getattr(network,"degree")()
    Similarity_inverse_log_weighted =getattr(network,"similarity_inverse_log_weighted")(vertices=None, mode=3)
    Similarity_dice =getattr(network,"similarity_dice")(vertices=None, pairs=None, mode=3, loops=True)
    Assortativity =getattr(network,"assortativity")(types1=getattr(network,"vs")["sex"], types2=None, directed=True)
    Assortativity_degree =getattr(network,"assortativity_degree")(directed=True)
    Assortativity_nominal =getattr(network,"assortativity_nominal")(types=getattr(network,"vs")["sex"], directed=True)
    Sex =network.vs["sex"]
    return  Degree, Betweenness, Transitivity_avglocal_undirected, Transitivity_undirected,\
            Transitivity_local_undirected, Transitivity_local_undirected, Diameter, Density, Diversity, Triad_census,\
            Vertex_connectivity, Similarity_jaccard, Similarity_inverse_log_weighted, Similarity_dice, Assortativity,\
            Assortativity_degree, Assortativity_nominal,Sex

def get_network_communities(network):
    """
    IN: Network object
    OUT: Detected communities through different algorithms
    """
    Community_infomap =getattr(network,"community_infomap")(edge_weights=None, vertex_weights=None, trials=10)
    Community_leading_eigenvector =getattr(network,"community_leading_eigenvector")(clusters=None, weights=None, arpack_options=None)
    Community_label_propagation =getattr(network,"community_label_propagation")(weights=None, initial=None, fixed=None)
    Community_edge_betweenness =getattr(network,"community_edge_betweenness")( clusters=None, directed=True, weights=None)
    Community_spinglass =getattr(network,"community_spinglass")(weights=None, spins=25, parupdate=False, start_temp=1, stop_temp=0.01, cool_fact=0.99, update_rule="config", gamma=1, implementation="orig", lambda_=1)
    Community_walktrap =getattr(network,"community_walktrap")( weights=None, steps=4)
    return  Community_infomap, Community_leading_eigenvector, Community_label_propagation, \
            Community_edge_betweenness, Community_spinglass, Community_walktrap


In [159]:
### Generates parameter for the synthetic networks ###

file_ids = [ int(fname.split("_")[0]) for fname in os.listdir(DATA_PATH) if fname.endswith('sex.csv')]
Degree_Collect=[]
Sex_Collect=[]
for file_id in file_ids:
    friend_graph,trust_graph = LoadData2Network(file_id)
    Degree, Betweenness, Transitivity_avglocal_undirected, Transitivity_undirected,\
    Transitivity_local_undirected, Transitivity_local_undirected, Diameter, Density, Diversity, Triad_census,\
    Vertex_connectivity, Similarity_jaccard, Similarity_inverse_log_weighted, Similarity_dice, Assortativity,\
    Assortativity_degree, Assortativity_nominal,Sex = get_network_param(trust_graph)
    Degree_Collect.append(Degree)
    Sex_Collect.append(Sex)

class_strength= [len(Degree_Collect[i]) for i in range(len(Degree_Collect))]
mean_degree=[np.sum(Degree_Collect[i])/class_strength[i] for i in range(len(Degree_Collect))]
p_values=[(mean_degree[i]/class_strength[i]) for i in range(len(avg_degree))]

In [165]:
### Generates synthetic statistics using the Erdos-Renyi random network ###
for n,p,s in zip(class_strength,p_values,Sex_Collect):
    null_model = Graph.Erdos_Renyi(n,p,directed=True)
    null_model.vs['sex']=s
    Degree, Betweenness, Transitivity_avglocal_undirected, Transitivity_undirected,\
    Transitivity_local_undirected, Transitivity_local_undirected, Diameter, Density, Diversity, Triad_census,\
    Vertex_connectivity, Similarity_jaccard, Similarity_inverse_log_weighted, Similarity_dice, Assortativity,\
    Assortativity_degree, Assortativity_nominal,Sex = get_network_param(null_model)